In [17]:
import pandas as pd
import geopandas as gpd
import xarray as xr
import sys, os
import time
import dask
import multiprocessing
ncpus = multiprocessing.cpu_count()
#from BurnCube import BurnCube #including burn mapping main functions
#bc = BurnCube()

from datacube_stats.statistics import GeoMedian

import datacube
from datacube.storage import masking
#from datacube.storage.masking import mask_to_dict
from datacube.storage.storage import write_dataset_to_netcdf
from datacube.helpers import ga_pq_fuser
dc = datacube.Datacube(app='dc_burnmap_comp')

In [18]:
outputdir = '/g/data/r78/DPIPWE_lm/test_burn_mapping/output_data'
if not os.path.exists(outputdir):
    print("output directory doesn't exist")
    exit()

In [19]:
subset = True
label = '12,-47'

len(label)

6

In [20]:
albers = gpd.read_file('/g/data/r78/DPIPWE_lm/test_burn_mapping/reference_data/Albers_Australia_Coast_Islands_Reefs.shp')

In [21]:
if label:
    index = albers[albers['label']==label].index[0]
    x = (albers.loc[index]['X_MIN'], albers.loc[index]['X_MAX'])
    y = (albers.loc[index]['Y_MIN'], albers.loc[index]['Y_MAX'])
    output_filename = outputdir + '/composite_2016-2017_'+'_'.join(label.split(','))+'.nc'
    print("Working on tile {}...".format(label))
else:
    x, y = (1385000.0, 1375000.0), (-4570000.0, -4580000.0)
    if subset:
        output_filename = 'composite_2016-2017_test_subset.nc'
    else:
        output_filename = 'composite_2016-2017_test_one.nc'

Working on tile 12,-47...


In [22]:
sensor = 'ls8'#_nbart_albers'
#datatime = ('2017-01-01', '2017-01-30') # period to retrieve data
#referenceperiod = ('2013-01-01', '2016-06-30') # period used for the calculation of geometric median
#mappingperiod = ('2016-07-01', '2017-06-30') # period of interest for change/severity mapping
#res = (25, 25)

query = {'x': x,
         'y': y,
         'time': ('2017-01-01', '2017-01-30'),
         'resolution': (250,250),
         'crs': 'EPSG:3577'}

In [23]:
#start_time = time.monotonic()
#try:
#    data = bc.load_cube(x, y, res, datatime, [sensor])
#except:
#    print("Problem loading data")
#    exit()
#print("---{} minutes for loading data.---".format((time.monotonic()-start_time)/60))

In [24]:
# Load PQ data for same query used to load Landsat data
#pq_ds


In [25]:
def burncomp(x, y):
    ds = dc.load(product=sensor+'_nbart_albers', group_by = 'solar_day', dask_chunks={'time': 1}, **query)
    
    # Load PQ data for same query used to load Landsat data
    pq_ds = dc.load(product = sensor+'_pq_albers',
                    group_by = 'solar_day',
                    fuse_func=ga_pq_fuser,
                    dask_chunks={'time': 1},
                    **query)

    # Use PQ to create mask that is True for pixels that are not affected by clouds, cloud shadow or saturation
    good_quality_ds = masking.make_mask(pq_ds.pixelquality,
                                    cloud_acca='no_cloud',
                                    cloud_fmask='no_cloud',
                                    cloud_shadow_acca='no_cloud_shadow',
                                    cloud_shadow_fmask='no_cloud_shadow',
                                    blue_saturated=False,
                                    green_saturated=False,
                                    red_saturated=False,
                                    nir_saturated=False,
                                    swir1_saturated=False,
                                    swir2_saturated=False,
                                    contiguous=True)

    # Remove -999 nodata values prior to analysing or plotting Landsat imagery by setting all nodata values to `NaN`
    ds = masking.mask_invalid_data(ds)

    # Apply the mask to preserve only the good data
    ds = ds.where(good_quality_ds) 
    
    # compute geomedian
    out = GeoMedian().compute(ds)
    return out.copy()

In [26]:
#geomedian_ds

In [27]:
xm, ym = (x[0]+x[1])/2, (y[0]+y[1])/2
x1, x2 = (x[0], xm), (xm, x[1])
y1, y2 = (y[0], ym), (ym, y[1])
if subset:
    out1 = burncomp(x1, y)
    #out2 = burncomp(x2, y)
    #out = xr.concat([out1, out2], dim='x')
else:
    out1 = burnmap(x, y)

In [28]:
out1

<xarray.Dataset>
Dimensions:          (x: 400, y: 400)
Coordinates:
  * y                (y) float64 -4.7e+06 -4.7e+06 ... -4.6e+06 -4.6e+06
  * x                (x) float64 1.2e+06 1.2e+06 1.201e+06 ... 1.3e+06 1.3e+06
Data variables:
    coastal_aerosol  (y, x) float64 nan nan nan nan ... 181.8 269.0 191.0 210.0
    blue             (y, x) float64 nan nan nan nan ... 142.8 226.0 150.0 172.2
    green            (y, x) float64 nan nan nan nan ... 239.4 357.0 252.5 307.1
    red              (y, x) float64 nan nan nan nan ... 265.4 342.0 230.0 272.3
    nir              (y, x) float64 nan nan nan ... 1.864e+03 2.234e+03
    swir1            (y, x) float64 nan nan nan ... 1.306e+03 904.0 1.048e+03
    swir2            (y, x) float64 nan nan nan nan ... 441.7 626.0 425.5 515.3
Attributes:
    crs:      EPSG:3577

In [39]:
# geomedian_ds = GeoMedian().compute(out)

# Output to netcdf
datacube.storage.storage.write_dataset_to_netcdf(out1, '/g/data/r78/DPIPWE_lm/test_burn_mapping/output_data/out_test_oct.nc')
#out1.geomed.to_netcdf('out_test_oct.nc')

AttributeError: 'Datacube' object has no attribute 'storage'